# Relatório de desenvolvimento do módulo `doc_data` versão `0.5.7`
**Autor**: Caio Geraldes

## Introdução

O módulo `doc_data`, escrito em `Python`, tem como função auxiliar a coleta dos dados necessários para a investigação proposta em meu doutorado.
O módulo se aproveita do corpus anotado `Diorisis` e do pacote de processamento de linguagem natural `stanza` para produzir árvores de dependência que filtram sentenças em que seria plausível encontrar atração de caso de predicados secundários de orações infinitivas do grego antigo.
Os detalhes de funcionamento do módulo está especificado no repositório no GitHub [caiogeraldes/doc_data](www.github.com/caiogeraldes/doc_data) e nos submódulos, funções e classes que compõem o projeto.

# Metodologia

O módulo `doc_data` segue, na versão `0.5.7`, as seguintes etapas para filtrar os dados:

1. Os textos disponibilizados no corpus `Diorisis` são separados em um ficheiro próprio;
2. Os textos são processados para retirar apenas os tokens de texto e metadata (autoria, documento etc);
3. Os tokens são reprocessados pelo modelo treinado com o corpus `Perseus` com ajuda do pacote `stanza`, produzindo sentenças e seus constituintes tokenizados com relações de dependência, análise morfológica e indexação;
4. Os tokens são armazenados em um banco de dados `NoSQL`, neste caso um `MongoDB`, para permitir filtragens mais precisas;
5. Filtram-se os tokens:
    - que pertençam a sentenças contendo um verbo principal de interesse (`mvi` = *main verb of interest*), selecionados em [mvi.csv](https://github.com/caiogeraldes/doc_data/blob/main/data/mvi.csv);
    - que pertençam a sentenças contendo um infinitivo dependente do `mvi`;
    - que pertençam a sentenças contendo um dativo ou genitivo dependente do `mvi`.
    
## Efeitos da filtragem

O código abaixo permite inspecionar o resultado da filtragem.

In [1]:
from doc_data.main import MONGO
from doc_data.db import mongo
from doc_data.tools import gen_sent, gen_meta, gen_hits
db = mongo(MONGO)
tokens = db.tokens
interest_tokens = db.interest_tokens
print(len(tokens.distinct("text-sentence")))
print(tokens.count_documents({}))
print(len(interest_tokens.distinct("text-sentence")))
print(interest_tokens.count_documents({}))

Connected successfully
92551
2855303
3621
199656


O corpus inicial `tokens` contém $92.551$ sentenças contendo $2.855.303$ tokens.
Após os três passos de filtragem, o corpus final `interest_tokens`, contém $3.621$ sentenças contendo $199.656$ tokens, $3.9\%$ do número de sentenças original ($7\%$ do número de tokens original).

## Qualidade da filtragem

**PÉSSIMA**

Seleciono aleatoriamente abaixo 36 sentenças ($\simeq 1\%$ do total) filtradas para averiguar se o método de filtragem produziu dados utilizáveis (a `seed` 36 foi escolhida arbitrariamente para que sempre se produzam as mesmas frases):

In [4]:
import random
random.seed(36)
xobj = db.xobjquery
tss = xobj.distinct("text-sentence")
for ts in random.choices(tss, k=36):
    print(gen_meta(interest_tokens, ts))
    print(gen_hits(interest_tokens, ts, [db.mviquery, db.infquery, xobj]))
    print(gen_sent(interest_tokens, ts))
    print()

Plato (0059) - Laws (034)
{'mviquery': ['δοκῇ', 'δοκεῖ'], 'infquery': ['πειρᾶσθαι', 'τρέπειν', 'δρᾶν'], 'xobjquery': ['ψυχαῖς', 'φιλοχρηματίᾳ', 'φιλονικίᾳ']}
ἄν τις δοκῇ πειρᾶσθαι τὴν τῶν δικαίων δύναμιν ἐν ταῖς τῶν δικαστῶν ψυχαῖς ἐπὶ τἀναντία τρέπειν καὶ παρὰ καιρὸν πολυδικεῖν τῶν τοιούτων ἢ καὶ συνδικεῖν , γραφέσθω μὲν ὁ βουλόμενος αὐτὸν κακοδικίας ἢ καὶ συνδικίας κακῆς , κρινέσθω δὲ ἐν τῷ τῶν ἐκλεκτῶν δικαστηρίῳ , ὀφλόντος δέ , τιμάτω τὸ δικαστήριον εἴτε φιλοχρηματίᾳ δοκεῖ δρᾶν τὸ τοιοῦτον εἴτε φιλονικίᾳ , καὶ ἐὰν μὲν φιλονικίᾳ , τιμᾶν αὐτῷ τὸ δικαστήριον ὅσου χρὴ χρόνου τὸν τοιοῦτον μηδενὶ λαχεῖν δίκην μηδὲ συνδικῆσαι , ἐὰν δὲ φιλοχρηματίᾳ , τὸν μὲν ξένον ἀπιόντα ἐκ τῆς χώρας μήποτε πάλιν ἐλθεῖν ἢ θανάτῳ ζημιοῦσθαι , τὸν ἀστὸν δὲ τεθνάναι φιλοχρημοσύνης ἕνεκα τῆς ἐκ παντὸς τρόπου παρ’ αὐτῷ τιμωμένης ·

Herodotus (0016) - Histories (001)
{'mviquery': ['ἐνετείλατο'], 'infquery': ['θάψαι'], 'xobjquery': ['ἑωυτῷ']}
Δαρεῖος δὲ πυθόμενος ταῦτα καὶ ἐπαιτιησάμενος τοὺς ταῦτα ποιήσαντας ὅτ